# Setup Environment and Download Data
## BƯỚC 1: Clone code và cài đặt package

In [ ]:
import os
import sys

# Clone repository nếu chưa có
if not os.path.exists('python_imageNet'):
    !git clone -b vu https://github.com/CodeWithVu/python_imageNet.git
    print("✅ Cloned repository")
else:
    print("✅ Repository already exists")

# Chuyển vào thư mục project
%cd python_imageNet

# Cài đặt package editable
!pip install -q -e .
print("✅ Package installed")

## BƯỚC 2: Upload và cấu hình Kaggle API
### Hướng dẫn lấy kaggle.json:
1. Vào https://www.kaggle.com
2. Settings → API → "Create New API Token"
3. Tải file `kaggle.json` về máy
4. Chạy cell dưới đây và upload file kaggle.json khi được yêu cầu

In [ ]:
# Upload kaggle.json từ máy local (qua VS Code)
from google.colab import files

print("📤 Vui lòng upload file kaggle.json...")
uploaded = files.upload()

# Setup Kaggle credentials
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
print("✅ Kaggle API configured")

## BƯỚC 3: Tải dataset AID từ Kaggle (chạy trên cloud)

In [ ]:
# Kiểm tra nếu data đã tải rồi thì skip
if not os.path.exists('./dataset/AID'):
    print("📥 Đang tải dataset từ Kaggle...")
    !kaggle datasets download -d cyian/aid-scene-classification-datasets -p ./dataset
    
    print("📦 Đang giải nén...")
    !unzip -q ./dataset/aid-scene-classification-datasets.zip -d ./dataset
    
    print("✅ Dataset ready!")
else:
    print("✅ Dataset already exists")

# Set đường dẫn data
sdir = './dataset/AID'
print(f"📁 Data path: {sdir}")

---
# Main Training Pipeline
## Import các module cần thiết

In [ ]:
from doan.data_loading import create_dataframe
from doan.data_analysis import analyze_dataset
from doan.data_trimming import trim_dataframe
from doan.generators import create_generators
from doan.model_creation import create_model
from doan.callbacks_setup import create_callbacks
from doan.training import train_model
from doan.visualization import plot_training_data, show_image_samples
from doan.evaluation import predictor, save_model

## Load và phân tích dataset

In [ ]:
# Load dataset
df = create_dataframe(sdir)
print(f'📊 Dataset loaded: {df.shape}')

# Analyze dataset
classes, class_count = analyze_dataset(df)
print(f'🏷️  Found {class_count} classes')
print(f'Classes: {classes[:5]}... (showing first 5)')

## Cân bằng dữ liệu (Data Trimming)

In [ ]:
# Trim dataset để balance
max_samples = 500  # Số samples tối đa mỗi class
min_samples = 50   # Số samples tối thiểu mỗi class
df_trimmed = trim_dataframe(df, max_samples, min_samples, 'labels')
print(f'✂️  Dataset after trimming: {df_trimmed.shape}')

## Chia dataset và tạo generators

In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset
train_df, temp_df = train_test_split(df_trimmed, test_size=0.3, random_state=42, stratify=df_trimmed['labels'])
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['labels'])

print(f'🔀 Train: {len(train_df)}, Valid: {len(valid_df)}, Test: {len(test_df)}')

# Create generators
img_size = (224, 224)
batch_size = 32
train_gen, valid_gen, test_gen, classes, class_count, test_steps = create_generators(train_df, valid_df, test_df, img_size, batch_size)
print(f'✅ Generators created')

## Tạo mô hình EfficientNetB3

In [ ]:
# Create EfficientNetB3 model
model = create_model(img_size + (3,), class_count)
model.summary()

## Huấn luyện mô hình

In [ ]:
# Setup callbacks
epochs = 20
ask_epoch = 10
callbacks = create_callbacks(model, epochs, ask_epoch)

# Train model
print("🚀 Starting training...")
history = train_model(model, train_gen, valid_gen, epochs, callbacks)
print("✅ Training completed!")

## Visualization

In [ ]:
# Plot training history
plot_training_data(history, 0)

# Show sample images
show_image_samples(train_gen)

## Đánh giá và lưu mô hình

In [ ]:
# Predict và evaluate
errors, tests = predictor(model, test_gen, test_steps, classes)

# Save model to Colab (hoặc Google Drive nếu đã mount)
save_dir = '/content/models'  # Hoặc '/content/drive/MyDrive/DoAn_Models' nếu mount Drive
os.makedirs(save_dir, exist_ok=True)

# Thay accuracy thực tế sau khi test
save_model(model, 'efficientnet_b3_aid', 95.5, save_dir)
print(f"✅ Model saved to {save_dir}")